In [1]:
import pdal
import json

In [8]:
pipeline = [
        {
            "type": "readers.las",
            "filename": f'data/sample/MDS_color_3316-221.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"ReturnNumber[1:4],NumberOfReturns![1:1],UserData[3:],Intensity[:10],Classification[19:19]"
        },
        {
            "type":"filters.voxelcentroidnearestneighbor",
            "cell":0.50
        },
        {
            "type":"filters.outlier",
            "method":"radius",
            "radius":2.0,
            "min_k":4
        },
        {
            "type":"filters.range",
            "limits":"Classification![7:7]"
        },
        {
            "type":"writers.las",
            "filename": f'results/sample/MDS_3316-221_1000-filtered.laz'
        },
        {
            "type":"writers.text",
            "format":"csv",
            "order":"X,Y",
            "keep_unspecified":"false",
            "filename":"results/sample/MDS_3316-221_1000-filtered.csv"
        }
        
    ]

In [9]:
filtered = pdal.Pipeline(json.dumps(pipeline))
n_points = filtered.execute()

In [10]:
## TODO
# Abrir o CSV
# Gerar um buffer de 2.5m
# Gerar o skelton
    ## https://github.com/AndaSampa/rotas-em-calcadas/blob/main/01%20-%20Rede%20de%20rotas%20caminhaveis%20um%20distrito%20-%20Vila%20Mariana.ipynb
# Resolver a intersecção com a vegetação
    ## Criar pontos CSV
    ## Gerar um buffer de 2.5
    ## Fazer a intersecção das linhas